In [17]:
#Installing nats_bench
!python3 -m pip install nats_bench
!python3 -m pip install gputil

In [18]:
#Installing AutoDL
!git clone https://github.com/D-X-Y/AutoDL-Projects
%cd /content/AutoDL-Projects/
!python3 -m pip install .

Cloning into 'AutoDL-Projects'...
remote: Enumerating objects: 6232, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 6232 (delta 31), reused 87 (delta 27), pack-reused 6133
Receiving objects: 100% (6232/6232), 11.24 MiB | 10.41 MiB/s, done.
Resolving deltas: 100% (4386/4386), done.
Updating files: 100% (457/457), done.
/content/AutoDL-Projects
Processing /content/AutoDL-Projects
  Preparing metadata (setup.py) ... done
  Created wheel for xautodl: filename=xautodl-1.0.0-py3-none-any.whl size=225892 sha256=ba0d6ab388f1f12e7e47502cfae28bad86f1356bb8c9159e4ffbc299381d97b7
  Stored in directory: /root/.cache/pip/wheels/9d/17/85/a4f85cb6a0cdf4223ebaf110795c95aea3177fd868bb6093c6
Successfully built xautodl
  Attempting uninstall: xautodl
    Found existing installation: xautodl 1.0.0
    Uninstalling xautodl-1.0.0:
      Successfully uninstalled xautodl-1.0.0


In [19]:
from google.colab import drive
from nats_bench import create
from nats_bench.api_utils import time_string
import numpy as np
import torch
import xautodl
from xautodl.models import get_cell_based_tiny_net
from xautodl.utils import count_parameters_in_MB
import platform
import GPUtil
import time
from tqdm import tqdm
import pandas as pd
import os

In [21]:
#Mounting drive location for NATS-Bench files
drive.mount("/content/gdrive", force_remount=True)
!ls /content/gdrive/MyDrive/SMR-Project/NATS-Bench-1/
%cd /content/gdrive/MyDrive/SMR-Project
!mkdir /content/gdrive/MyDrive/SMR-Project/results
%cd NATS-Bench-1


Mounted at /content/gdrive
AutoDL-Projects			 NATS-sss-v1_0-50262-simple.tar   NATS-tss-v1_0-3ffb9-simple.tar
cifar.python			 NATS-tss-v1_0-3ffb9.pickle.pbz2
NATS-sss-v1_0-50262.pickle.pbz2  NATS-tss-v1_0-3ffb9-simple
/content/gdrive/.shortcut-targets-by-id/1m_5Kr7rURbHKXCurFDFRHAUYTXc8MVsc/SMR-Project
mkdir: cannot create directory ‘/content/gdrive/MyDrive/SMR-Project/results’: File exists
/content/gdrive/.shortcut-targets-by-id/1m_5Kr7rURbHKXCurFDFRHAUYTXc8MVsc/SMR-Project/NATS-Bench-1


In [9]:
!cp -r NATS-tss-v1_0-3ffb9-simple /root/.torch
# !cp -r NATS-sss-v1_0-50262-simple /root/.torch

^C


In [22]:
api = create('NATS-tss-v1_0-3ffb9-simple', 'tss', fast_mode=True, verbose=True)
print(len(api.meta_archs))

[2023-10-15 02:44:56] Try to create the NATS-Bench (topology) api from NATS-tss-v1_0-3ffb9-simple with fast_mode=True
[2023-10-15 02:44:56] Create NATS-Bench (topology) done with 0/15625 architectures avaliable.


In [23]:

def cuda_time() -> float:
    torch.cuda.synchronize()
    return time.perf_counter()
@torch.inference_mode()
def measure(model, img_size, num_repeats=500, num_warmup=500):
    # model.cuda()
    model.eval()

    # backbone = model.backbone
    inputs = torch.randn(4, 3, img_size, img_size).cuda()

    latencies = []
    for k in range(num_repeats + num_warmup):
        start = cuda_time()
        model(inputs)
        if k >= num_warmup:
            latencies.append((cuda_time() - start) * 1000)

    #latencies = itertools.chain(dist.allgather(latencies))
    latencies = sorted(latencies)

    drop = int(len(latencies) * 0.25)
    return np.mean(latencies[drop:-drop])

In [39]:
def latency(api_main,i,dataset='cifar10'):
  config = api.get_net_config(i, dataset)
  network = get_cell_based_tiny_net(config)
  info = api.get_more_info(i, dataset)
  cost = api.get_cost_info(i,dataset)
  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  network = network.to(device)
  if(dataset == 'ImageNet16-120'):
    img_size = 256
  else:
    img_size = 32
  latency = measure(network,img_size)
  data=[config['name'],config['C'],config['N'],config['arch_str'],info['test-accuracy'],info['test-all-time'],info['test-per-time'],info['train-accuracy'],info['train-all-time'],info['train-per-time'],cost['flops'],cost['latency'], cost['params'],cost['T-ori-test@epoch'], cost['T-ori-test@total'], cost['T-train@epoch'], cost['T-train@total'], latency]
  return data




In [46]:
# Create the API for size search space
root_dir = '/content/gdrive/MyDrive/SMR-Project/results'
if torch.cuda.is_available():
  hw = str(GPUtil.getGPUs()[0].name)
else:
  uname = platform.uname()
  hw = str(uname.machine) +'_'+str(uname.processor)
if(not(os.path.exists(os.path.join(root_dir,hw)))):
  os.makedirs(os.path.join(root_dir,hw))

latency_cifar10 = []
latency_cifar100 = []
latency_imagenet = []

total_models = len(api.meta_archs)
# total_models = 4
for i in tqdm(range(total_models),total=total_models):
  latency_cifar10.append(latency(api,i,'cifar10'))
  latency_cifar100.append(latency(api,i,'cifar100'))
  latency_imagenet.append(latency(api,i,'ImageNet16-120'))









  0%|          | 0/4 [00:00<?, ?it/s]

[2023-10-15 03:00:55] Call the get_net_config function with index=0, dataset=cifar10.
[2023-10-15 03:00:55] Call _prepare_info with index=0 skip because it is in arch2infos_dict
[2023-10-15 03:00:55] Call the get_more_info function with index=0, dataset=cifar10, iepoch=None, hp=12, and is_random=True.
[2023-10-15 03:00:55] Call query_index_by_arch with arch=0
[2023-10-15 03:00:55] Call _prepare_info with index=0 skip because it is in arch2infos_dict
[2023-10-15 03:00:55] Call the get_cost_info function with index=0, dataset=cifar10, and hp=12.
[2023-10-15 03:00:55] Call _prepare_info with index=0 skip because it is in arch2infos_dict
Call query_meta_info_by_index with arch_index=0, hp=12
[2023-10-15 03:00:55] Call _prepare_info with index=0 skip because it is in arch2infos_dict
[2023-10-15 03:01:01] Call the get_net_config function with index=0, dataset=cifar100.
[2023-10-15 03:01:01] Call _prepare_info with index=0 skip because it is in arch2infos_dict
[2023-10-15 03:01:01] Call the g

 25%|██▌       | 1/4 [00:35<01:47, 35.85s/it]

[2023-10-15 03:01:31] Call the get_net_config function with index=1, dataset=cifar10.
[2023-10-15 03:01:31] Call _prepare_info with index=1 skip because it is in arch2infos_dict
[2023-10-15 03:01:31] Call the get_more_info function with index=1, dataset=cifar10, iepoch=None, hp=12, and is_random=True.
[2023-10-15 03:01:31] Call query_index_by_arch with arch=1
[2023-10-15 03:01:31] Call _prepare_info with index=1 skip because it is in arch2infos_dict
[2023-10-15 03:01:31] Call the get_cost_info function with index=1, dataset=cifar10, and hp=12.
[2023-10-15 03:01:31] Call _prepare_info with index=1 skip because it is in arch2infos_dict
Call query_meta_info_by_index with arch_index=1, hp=12
[2023-10-15 03:01:31] Call _prepare_info with index=1 skip because it is in arch2infos_dict
[2023-10-15 03:01:38] Call the get_net_config function with index=1, dataset=cifar100.
[2023-10-15 03:01:38] Call _prepare_info with index=1 skip because it is in arch2infos_dict
[2023-10-15 03:01:38] Call the g

 50%|█████     | 2/4 [01:29<01:33, 46.60s/it]

[2023-10-15 03:02:25] Call the get_net_config function with index=2, dataset=cifar10.
[2023-10-15 03:02:25] Call _prepare_info with index=2 skip because it is in arch2infos_dict
[2023-10-15 03:02:25] Call the get_more_info function with index=2, dataset=cifar10, iepoch=None, hp=12, and is_random=True.
[2023-10-15 03:02:25] Call query_index_by_arch with arch=2
[2023-10-15 03:02:25] Call _prepare_info with index=2 skip because it is in arch2infos_dict
[2023-10-15 03:02:25] Call the get_cost_info function with index=2, dataset=cifar10, and hp=12.
[2023-10-15 03:02:25] Call _prepare_info with index=2 skip because it is in arch2infos_dict
Call query_meta_info_by_index with arch_index=2, hp=12
[2023-10-15 03:02:25] Call _prepare_info with index=2 skip because it is in arch2infos_dict
[2023-10-15 03:02:32] Call the get_net_config function with index=2, dataset=cifar100.
[2023-10-15 03:02:32] Call _prepare_info with index=2 skip because it is in arch2infos_dict
[2023-10-15 03:02:32] Call the g

 75%|███████▌  | 3/4 [02:24<00:50, 50.04s/it]

[2023-10-15 03:03:19] Call the get_net_config function with index=3, dataset=cifar10.
[2023-10-15 03:03:19] Call _prepare_info with index=3 skip because it is in arch2infos_dict
[2023-10-15 03:03:19] Call the get_more_info function with index=3, dataset=cifar10, iepoch=None, hp=12, and is_random=True.
[2023-10-15 03:03:19] Call query_index_by_arch with arch=3
[2023-10-15 03:03:19] Call _prepare_info with index=3 skip because it is in arch2infos_dict
[2023-10-15 03:03:19] Call the get_cost_info function with index=3, dataset=cifar10, and hp=12.
[2023-10-15 03:03:19] Call _prepare_info with index=3 skip because it is in arch2infos_dict
Call query_meta_info_by_index with arch_index=3, hp=12
[2023-10-15 03:03:19] Call _prepare_info with index=3 skip because it is in arch2infos_dict
[2023-10-15 03:03:23] Call the get_net_config function with index=3, dataset=cifar100.
[2023-10-15 03:03:23] Call _prepare_info with index=3 skip because it is in arch2infos_dict
[2023-10-15 03:03:23] Call the g

100%|██████████| 4/4 [02:49<00:00, 42.47s/it]


In [48]:
all_cols = ['model', 'C','N','arch_str','test-acc','test-all-time','test-per-time','train-acc','train-all-time','train-per-time','flops','latency-gen','params','T-ori-test@epoch','T-ori-test@total','T-train@epoch','T-train@total','latency']
df_cifar10 = pd.DataFrame(latency_cifar10,  columns=all_cols)
df_cifar100 = pd.DataFrame(latency_cifar100,  columns=all_cols)
df_imagenet = pd.DataFrame(latency_imagenet,  columns=all_cols)

In [49]:
df_cifar10.to_csv(os.path.join(root_dir,hw,'cifar10.csv'))
df_cifar100.to_csv(os.path.join(root_dir,hw,'cifar100.csv'))
df_imagenet.to_csv(os.path.join(root_dir,hw,'imagenet16.csv'))